## 本地运行大语言

*	硬件要求

        CPU：多核处理器（推荐 4 核或以上）。
        GPU：如果你计划运行大型模型或进行微调，推荐使用具有较高计算能力的 GPU（如 NVIDIA 的 CUDA 支持）。
        内存：至少 8GB RAM，运行较大模型时推荐 16GB 或更高。
        存储：需要足够的硬盘空间来存储预训练模型，通常需要 10GB 至数百 GB 的空间，具体取决于模型的大小。
        软件要求：确保系统上安装了最新版本的 Python（如果打算使用 Python SDK）

*	个人电脑(最好是Mac可以无缝使用GPU加速,windows也可以)

*   安装好python3

*   电脑安装[Ollama](https://ollama.com)

*   启动Ollama

        ollama serve

*   安装python模块

        pip install ollama

*   [模型下载](https://ollama.com/models),本地安装下载，示例代码：

        ollama run deepseek-r1

In [1]:
from ollama import chat, ChatResponse
import os

os.environ['OLLAMA_GPU_METAL'] = '10'
# 读取本地文件内容
def read_analysis_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

# 构建分析提示
def create_analysis_prompt(file_content):
    prompt = f"""
    请严格按照以下要求分析宏病毒样本数据：

    【数据开始】
     {file_content}
    【数据结束】

    【数据处理】
     行间比较，如果具有相同Species但不同Genotype或Isolate的认为重复行,选取其中一行作为代表,选取标准如下：
       - Avg_fold(平均覆盖深度)：越高表示该物种丰度越高
       - Covered_percent(覆盖百分比)：越高表示序列完整性越好
       - Covered_bases(覆盖碱基数):越高越好
       - Median_fold(中位覆盖深度):越高越好
       - Plus_reads/Minus_reads:比对到Species正负链reads数

    [输出格式]
    禁止输出思考过程,请直接给出分析结果，严格遵循以下格式
    |物种名称|Avg_fold |Covered_percent | Median_fold｜Plus_reads | Minus_reads |
    """
    print(prompt)
    return prompt

# 主分析函数
def analyze_metagenomics_sample(file_path):
    # 读取文件内容
    file_content = read_analysis_file(file_path)

    # 创建分析提示
    prompt = create_analysis_prompt(file_content)

    # 使用Ollama进行聊天分析
    response: ChatResponse = chat(
        model='deepseek-r1:latest',  # 或者你本地安装的其他模型，如 'mistral', 'qwen' 等
        messages=[
            {
                'role': 'user',
                'system': '直接回答，不要思考过程',
                'content': prompt
            }
        ],
        options = {
        'temperature': 0.0,  # Set to 0.0 for maximum determinism
        'seed': 42  # Use a specific integer seed (e.g., 42, 123)
        }
    )

    return response['message']['content']

# 使用示例
if __name__ == "__main__":
    file_path = "./test.txt"  # 你的文件路径
    result = analyze_metagenomics_sample(file_path)
    print("分析结果：")
    print(result)


    请严格按照以下要求分析宏病毒样本数据：

    【数据开始】
     #ID	Avg_fold	Length	Ref_GC	Covered_percent	Covered_bases	Plus_reads	Minus_reads	Read_GC	Median_fold	Std_Dev
PQ428138.1|Lymphocryptovirus humangamma4 isolate AA-65, complete genome	56.6403	168915	0.0000	84.7764	143200	31736	32602	0.5573	44
MK973062.1|Human gammaherpesvirus 4 strain rMSHJ, complete genome	58.3010	164674	0.0000	90.2705	148652	32038	32432	0.5575	46
PQ428130.1|Lymphocryptovirus humangamma4 isolate AA-56, complete genome	55.8287	171706	0.0000	85.5124	146830	31850	32731	0.5576	43
PV296186.1|Coxsackievirus A6 isolate APK2242/NJ/CHN/2022, complete genome	3458.2821	7430	0.0000	100.0000	7430	87782	89069	0.4841	2836
LC573552.1|Human gammaherpesvirus 4 sLCL-T12.18 DNA, complete genome	54.3878	183865	0.0000	80.7370	148447	33451	33770	0.5594	39
NC_030454.1|Enterovirus A114 strain V13-0285, partial genome	1791.8781	7317	0.0000	66.6940	4880	45030	44488	0.4891	776
AJ507799.2|Human herpesvirus 4 complete wild type genome	55.9405	171823	0.0000	86.

KeyboardInterrupt: 